In [1]:
import requests
import pyspark.sql.functions as f
from pyspark.sql.functions import udf,col
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession

### 1. Create spark session

In [2]:
# Create a Spark session
spark_1 = SparkSession.builder.appName("1ForwardDataAnalysis_new").master("local[*]").config("spark.driver.memory", "15g").getOrCreate()

24/01/19 05:40:35 WARN Utils: Your hostname, cssh-alpha resolves to a loopback address: 127.0.1.1; using 134.130.186.164 instead (on interface bond0)
24/01/19 05:40:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/19 05:40:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/19 05:40:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# spark_1.stop()

### 2. Load dataset

In [4]:
# Specify the path to the folder containing Orc files
input_folder_origin = "/home/cxu/swh-popular-3k-python/origin_visit_status"
# Read Orc files into a DataFrame
origin = spark_1.read.format("orc").load(input_folder_origin)

# Data description
# Print the schema of the dataset
# revision.printSchema()
# Show the first 5 rows of the dataset
origin.show(1)
# Get the length of the dataset
# dataset_length = revision.count()
# Show the length of the dataset
# print(f"Length of the dataset: {dataset_length} rows")

+--------------------+-----+--------------------+-------+--------------------+
|              origin|visit|                date| status|            snapshot|
+--------------------+-----+--------------------+-------+--------------------+
|deb://Debian/pack...|  287|2017-12-22 16:34:...|partial|48cfa88294968d0de...|
+--------------------+-----+--------------------+-------+--------------------+
only showing top 1 row



In [5]:
origin.select('snapshot').distinct().count()

19382

### 3. Query

In [6]:
# git log: given a revision node, recursively list all linked nodes of type revision

# Endpoint:

# /graph/visit/nodes/:REV_ID?edges=rev:rev

In [7]:
snps = origin.select('snapshot').distinct().collect()

In [8]:
len(snps)

19382

In [9]:
# with open('snps.text', 'w') as f:
#     f.write('\n'.join(str(i['snapshot']) for i in snps))

In [10]:
part_1 = snps[:4000], 
part_2 = snps[4000:8000], 
part_3 = snps[8000:12000], 
part_4 = snps[12000:16000], 
part_5 = snps[16000:]
len(part_1),len(part_2),len(part_3),len(part_4),len(part_5),

(1, 1, 1, 1, 3382)

In [11]:
# from pyspark.sql.types import *
# df = spark.createDataFrame([], 
#                               schema=StructType(fields=[
#                                   StructField("snapshot", StringType()), 
#                                   StructField("revisions", ArrayType(StringType()))]))

In [12]:
# df.printSchema(), df.show()

In [13]:
# columns = df.columns
# columns

In [14]:
for index,snp in enumerate(snps[:4000]):
# Replace 'your-api-endpoint' and 'your-path' with the actual API endpoint and path
    # print(snp['snapshot'])
    # snp_rev = {}
    api_endpoint = "http://localhost:5009"
    print(index, snp)

    # Revison -> Origin X(Origin id abandon)
    path = f"/graph/visit/nodes/swh:1:snp:{snp['snapshot']}"
    response = requests.get(api_endpoint + path)
    text = response.text.splitlines()
    origins = [l for l in text if 'rev' in l]

    origin_snaps = snp['snapshot'] + ':' + str(origins)
    # print(origin_snaps)

    with open('result1.txt', 'a') as file:
        file.writelines(origin_snaps+'\n')


    # new_row = spark_forward.createDataFrame([(snp['snapshot'], origins)], columns)
    # snp_rev[snp['snapshot']] = [l for l in text if 'rev' in l]
    # print(snp_rev)
    # df = df.union(new_row)
    # df.write.mode('append').orc('result_1.orc')
    


0 Row(snapshot='e782789e19560aed3b6eaf9ed2847702013d9f19')
1 Row(snapshot='4d2dac5c607a2cacb4301d9344244a730297262c')
2 Row(snapshot='bc1a0babc0c8485a5625a75996d9a9dc4854b289')
3 Row(snapshot='0fbaa115e7fe163ce75a388cb56c4f80fb3f7465')
4 Row(snapshot='6b8f724a4fbd9a2c7a16cd8eb3e485434cf6c325')
5 Row(snapshot='da84a8b50ea49efd9756b213843de715becdbd86')
6 Row(snapshot='a15e803497a7da06f25f16cf9faa99933e848c46')
7 Row(snapshot='20270dbf7bd07a326115179af55acd479ae97585')
8 Row(snapshot='74d2a6faf8aaa9b6af74c6136c0389c3c7ce4d4a')
9 Row(snapshot='826cd41fdd676d3342d253f968c05bc3b878a1c1')
10 Row(snapshot='3612d51f656842079cd24927c1ed227792b7ea35')
11 Row(snapshot='67503f4e72c5c3bf48c7ebdb26908f289985b97e')
12 Row(snapshot='909d63c4bc806d66165750a4163159ce8bfaeecf')
13 Row(snapshot='452787b02a4ba64e535b63ade4c0179b20c8c252')
14 Row(snapshot='1418cc61e01f363d833cacab7477d5a8490e767a')
15 Row(snapshot='b3f2df6fc9f7c8943ccef4bf301ed38ea6fa963f')
16 Row(snapshot='31ed4aaf9e3e698526ac789061e6de4c5

19 Row(snapshot='54edd743cc086d3cfe4cd5e7f177926d939abe81')
20 Row(snapshot='c41921220e453e0bbb9ec354bff30a25d023c0b9')
21 Row(snapshot='409011c6542af44a2e6bcb6bc6d21d27a619df0b')
22 Row(snapshot='4a37e267661285e114f6516e9fee37506c4c71f0')
23 Row(snapshot='6647dc947706f57f357c23d0e7c311511a6a62ef')
24 Row(snapshot='dfcbba8aab5d7ddc4632ca031376bb627700b7d2')
25 Row(snapshot='4c7829bc337e63be97df4c675e9fa79befaf2afa')
26 Row(snapshot='da498c85621b49f9d348ba44c69fb84ac3f826ea')
27 Row(snapshot='39e1ac4f79c00993ae3146b1a33601707cac4e80')
28 Row(snapshot='fae4cf48675bcbc5e6b0ce2baad00973a82c0689')
29 Row(snapshot='421f51bb9fac16ef36eb43b6fc3fe348a7fb20b6')
30 Row(snapshot='203eb11608adc5d000fbe05eacc2d6f56d928a57')
31 Row(snapshot='685fc966c55f3391df6dab56283263e0e714e122')
32 Row(snapshot='47d9c5bb1f8d586c7318464bac21364f06e68125')
33 Row(snapshot='2657b2b9bd2fc694be5527b6c35c6afe7dcfe59c')
34 Row(snapshot='6f80224901cee8522621076ec60dcd18c91403a1')
35 Row(snapshot='c89306c02af718c314d5b29

In [ ]:
# spark_forward.createDataFrame(snp_rev.items()).show()

In [ ]:
# df.write.orc('result/')